앞서 했던 [제곱근 구하기](http://wikibootup.github.io/sicp/1-Abstraction-using-procedure/1-sqrt.html)를 돌이켜보면, 원하는 결과가 나올 때까지 '동일한 연산'을 계속 적용했다는 것을 알 수 있습니다.

연산은 함수 형태로 표현했고, 이전 함수의 결과가 다음 함수의 인자로 쓰이는 과정을 밟고 있습니다.

수식으로 표현하면,

### $f(f(f...(f(x)))...) = x$

이 때의 x를 f의 고정점fixed point이라고 합니다.

여기서는 이 방법을 이용해 연속분수 문제를 해결해보겠습니다.

연속 분수를 표현하면,

##$f = \frac{N_1}{D_1 + \frac{N_2}{D_2 + \frac{N_3}{D_3 + ...}}}$

위와 같습니다.

약간 복잡해보이지만 순서대로 분리해서 보면 간단해집니다.

첫번째 차례에 $\frac{N_1}{D_1}$으로 시작해서,

두번째 차례에 $\frac{N_2}{D_2}$를 첫번째 결과의 분모에 더해줍니다.

이후 과정은 두번째 차례와 동일한 방식으로 진행됩니다.

따라서 연속 분수는 동일한 연산을 반복하는 과정이므로

연속 분수의 값은 고정점 찾기로 구할 수 있습니다.

고정점을 찾는 함수가 있다고 가정하고,

문제를 하나 풀어보겠습니다.

아래는 연속 분수로 나타낸 탄젠트 함수입니다.

#$tan\ x = \frac{x}{1 - \frac{x^2}{3 - \frac{x^2}{5 - ...}}}$

연속 분수에서 고정점을 찾는 함수의 이름을 ```continued_fraction```으로 두고

이 문제를 푸는 코드를 작성하면,

In [1]:
def tan(x, k):
    return continued_fraction(
        lambda i: x if i == 1 else x ** 2,
        lambda i: 2 * i - 1,
        lambda op1, op2: op1 - op2,
        k
    )

만들어야 할 ```continued_fraction``` 함수는 다음과 같은 성질을 가집니다.

```
continued_fraction(n, d, op, k)
n: 분자값을 구하기 위한 함수
d: 분모값을 구하기 위한 함수
op: 분수 사이에 들어갈 연산
k: k번째까지의 분자값, 분모값이 있는 연속 분수
```

보통은 재귀보다 반복 프로시저가 구현하기 성가신데

연속분수는 반복 프로시저가 더 깔끔하게 나온다는 생각이 들었습니다.

그런 의미에서 반복 프로시저로 ```continued_fraction``` 함수를 구현하면,

In [2]:
def continued_fraction(n, d, op, k):
    def cf_iter(k, result):
        if k == 0:
            return result
        else:
            return cf_iter(k-1, n(k) / op(d(k), result))
    return cf_iter(k, 0)

\* 위의 구현은 k번째 분자와 분모 값을 구하는 것에서부터 시작합니다.

이 함수를 Python에서 제공하는 ```tan``` 함수와 대조해보면,

### $\frac{\pi}{6},\ \frac{\pi}{4},\ \frac{\pi}{3}$

In [6]:
import math

print(tan(math.pi / 6, 300))
print(math.tan(math.pi / 6))

print(tan(math.pi / 4, 300))
print(math.tan(math.pi / 4))

print(tan(math.pi / 3, 300))
print(math.tan(math.pi / 3))

0.5773502691896257
0.5773502691896256
1.0
0.9999999999999999
1.732050807568877
1.7320508075688767


$\frac{\pi}{2}$가 $\infty$로 발산하므로 이 근방을 피하여

범위를 $\frac{\pi}{8} < x < \frac{\pi}{3}$, $-\frac{\pi}{3} < x < -\frac{\pi}{8}$ 으로 설정하여

좀 더 대조해보면,

In [7]:
ALLOWABLE_ERROR = 0.00001

def accuracy_test(op1, op2, start, end, diff):
    if start > end:
        return True
    return abs(op1(start) - op2(start)) < ALLOWABLE_ERROR and\
        accuracy_test(op1, op2, start + diff, end, diff)

In [8]:
accuracy_test(lambda angle: tan(angle, 100), math.tan, math.pi/8, math.pi/3, 0.001)

True

In [12]:
accuracy_test(lambda angle: tan(angle, 100), math.tan, - math.pi / 3, - math.pi / 8, 0.001)

True

n을 100에서 5로 낮추어 시도해보면,

In [10]:
accuracy_test(lambda angle: tan(angle, 5), math.tan, math.pi/8, math.pi/3, 0.001)

True

In [13]:
accuracy_test(lambda angle: tan(angle, 5), math.tan, - math.pi / 3, - math.pi / 8, 0.001)

True

매우 적은 n 값으로도($n=5$) 파이썬 라이브러리에서 제공하는 $tan$ 값과 근사하게 다다감을 알 수 있습니다.

다음 글에서는 '프로그램을 만들 수 있는 조건에 대한 가정'에 대한 이야기와 그에 들어맞는 문제를 풀어보겠습니다.